In [4]:
import numpy as np
import matplotlib.pyplot as plt
import random as rand

In [15]:
def TableGenerator(MesString,AliBasString,BobBasString,MeasString):
    #Intput: 4 arrays of 02 or 1s of same length
    #Output: Merge them into a table
    
    Table = np.zeros([4,len(MesString)])
    Table[0,:] = MesString
    Table[1,:] = AliBasString
    Table[2,:] = BobBasString
    Table[3,:] = MeasString
    
    return Table

def TableShuffler(Table):
    #Input: Table, a 4xn_iter array
    #Output: Table Shuffled in columns (permutation)
    rng = np.random.default_rng()
    Shuffled = np.transpose(rng.permutation(np.transpose(Table)))
    return Shuffled

def TableFiltering(Table):
    #Input Table, a 4xn_iter array
    #Output: Table Filtered: Only those columns in which
    #Alice Basis = Bob Basis
    #i = 0: Alice Message
    #i = 1: Alice Basis
    #i = 2: Bob Basis
    #i = 3: Bob Measures
    SameBasisTable = np.zeros([4,1])
    for i in range(len(Table[0,:])):
        if Table[1,i] == Table[2,i]:
            newcol = np.zeros([4,1])
            for j in range(4):
                newcol[j,0]=Table[j,i]
            SameBasisTable = np.append(SameBasisTable,newcol,axis=1)
    SameBasisTable = np.delete(SameBasisTable,0,axis=1)
    return SameBasisTable

def TableSampling(Table,frac=0.1):
    #Extracts a sample to caclulate the raw probability of failure and expose Eve, if present
    #Input: Table to work upon, fraction of the columns devoted to sampling
    random_indx = np.random.randint(0,len(Table[0,:]),int(frac*len(Table[0,:])))
    SampleTable = Table[:,random_indx]
    
    sum = 0
    for i in range(len(SampleTable[0,:])):
        if SampleTable[0,i]==SampleTable[3,i]:
            sum +=1
    
    rawSuccessRate = sum/float(len(SampleTable[0,:]))
    
    newTable = Table
    newTable = np.delete(Table,random_indx,axis=1)
    
    return newTable, SampleTable, rawSuccessRate

def TableReorganisation(Table,n_rep):
    #Input: Filtered Table, number of iterations for repetition code
    #Output: Reorganised table by columns in groups of n_rep
    newTable = Table
    index_row = np.zeros(len(Table[0,:]),dtype = int)
    index_now = 0
    index_list = []
    for i in range(len(Table[0,:])):
        #print(i,"/",len(Table[0,:]))
        #print(index_row)
        if index_row[i] != 0:
            continue
        else:
            index_row[i] = index_now
            index_list.append(i)
            column_counter = 1
            leap = 1
            while column_counter < n_rep and i + leap < len(Table[0,:]): #We have not found all the columns or we have run out of columns
                if Table[0,i]==Table[0,i+leap] and Table[1,i]==Table[1,i+leap] and Table[1,i]==Table[1,i+leap]:
                    index_now += 1
                    column_counter += 1
                    index_row[i+leap] = index_now
                    index_list.append(i+leap)
                leap += 1
            index_now +=1
    
    #Table Reorganisation
    newTable[:] = Table[:,index_list]
    
    
    return newTable

def TableMajorityVoting(Table,n_rep):
    #Given the reorganised table, merges the columns applying majority voting
    newTable = np.zeros([4,1])
    i = 0
    leap = 0
    while i + leap < len(Table[0,:]):
        newcol = np.zeros([4,1])
        for j in range(3):
            newcol[j,0]=Table[j,i]
        leap = 0
        cache = []
        while i + leap < len(Table[0,:]) and Table[0,i]==Table[0,i+leap] and Table[1,i]==Table[1,i+leap] and  leap <n_rep:
            cache.append(Table[3,i+leap])
            leap += 1
        
        #Majority voting
        if len(cache) == n_rep:
            if cache.count(0) > cache.count(1):
                newcol[3,0] = 0
            else:
                newcol[3,0]= 1
        
            newTable = np.append(newTable,newcol,axis=1)
        i = i+leap
    
    newTable = np.delete(newTable,0,axis=1)
    
    return newTable

def NewStrings(Table):
    #Given the Mayority Voted Table, returns the new Alice Message and Bob Measures strings
    newAliceString = Table[0,:]
    newBobString = Table[3,:]
    
    return newAliceString,newBobString

def WholeRepetitionProcedure(AliceMess,AliceBas,BobBas,BobMeasure,n_rep,checkingForEve = False):
    #Input: The 4 strings obtained through BB84 or E91 protocols, and how many repetition bits n_rep
    # boolean argument "checkingForEve" set to be false by defect
    
    #Output: The new AliceKey and BobKey after the repetition protocol
    # If checking for Eve, also the success rate before applying majority voting
    
    Table = TableGenerator(AliceMess,AlicBas,BobBas,Measure)
    Shuffled = TableShuffler(Table)
    Filtered = TableFiltering(Shuffled)
    if checkingForEve:
        Sampled,Sample,rawSucRate = TableSampling(Filtered)
        Reorganised = TableReorganisation(Sampled,n_rep)
    else:
        Reorganised = TableReorganisation(Filtered,n_rep)

    Majority = TableMajorityVoting(Reorganised,n_rep)
    NewAliceKey, NewBobKey = NewStrings(Majority)
    
    if checkingForEve:
        return NewAliceKey,NewBobKey,rawSucRate
    else:
        return NewAliceKey,NewBobKey
        

In [24]:
#Trial
Gross_length = 40
n_rep = 3
Message = np.random.randint(2,size = Gross_length)
AlicBas = np.random.randint(2,size = Gross_length)
BobBas  = np.random.randint(2,size = Gross_length)
Measure = np.random.randint(2,size = Gross_length)

Table = TableGenerator(Message,AlicBas,BobBas,Measure)
print("Initial Table")
print(Table)
print()

Shuffled = TableShuffler(Table)
print("Shuffled")
print(Shuffled)
print()

Filtered = TableFiltering(Shuffled)
print("Filtered")
print(Filtered)
print()

Sampled, Sample, rawSuccessRate = TableSampling(Filtered)
print("Sampled")
print(Sampled)
print("Sample")
print(Sample)
print("Raw Success Rate:",rawSuccessRate)
print()

Reorganised = TableReorganisation(Sampled,n_rep)
print("Reorganised")
print(Reorganised)
print()

Majority = TableMajorityVoting(Reorganised,n_rep)
print("Majority")
print(Majority)
print()

NewAliceKey, NewBobKey = NewStrings(Majority)
print("New Keys")
print("Alice Key:", NewAliceKey)
print("Bob Key: ",NewBobKey)
print()

print("WholeProcedure without checking")
FinalAliceKey, FinalBobKey = WholeRepetitionProcedure(Message,AlicBas,BobBas,Measure,n_rep,False)
print("Alice Key:", FinalAliceKey)
print("Bob Key: ",FinalBobKey)
print()

print("WholeProcedure checking")
FinalAliceKey, FinalBobKey, rawSucRate = WholeRepetitionProcedure(Message,AlicBas,BobBas,Measure,n_rep,True)
print("Alice Key:", FinalAliceKey)
print("Bob Key: ",FinalBobKey)
print("Raw Success Rate:",rawSucRate)
print()

Initial Table
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0.
  0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1.
  0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1.
  0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.
  1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0.]]

Shuffled
[[1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0.
  1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1.
  0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0.
  0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.
  1. 0. 1. 0. 0. 0. 1. 1. 